In [25]:
from HH_Lukas import HouseholdSpecializationModelClass as HC
from types import SimpleNamespace
import numpy as np  
from scipy import optimize
import ipywidgets as widgets
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib import cm

%load_ext autoreload
%autoreload 2

## Question 1

### First, we solve for one set of parameters: 

In [26]:
a = HC() #Import the whole model
a.par.alpha = 0.25 #Set parameters
a.par.sigma = 0.5
a_1 = a.solve_discrete() #solve the model


#Get hours worked for females and males and compute ratio
HF_1 = a_1.HF  
HM_1 = a_1.HM
HFHM_1=HF_1/HM_1
print('The ratio of female to male hours worked at home is ', HFHM_1)


The ratio of female to male hours worked at home is  0.5454545454545454


### Set parameters to loop through

In [27]:
#Parameters
alphas = np.array([0.25, 0.5, 0.75])
sigmas = np.array([0.5, 1.0, 1.5])
#Initialize arrays for hours worked at home for males and females
HM = np.zeros(9) 
HF = np.zeros(9)

#### Indexing
Figure out an index for all nine possible combinations of the parameters. The combinations can be found using the Cartesian product of the "alphas" and "sigmas" arrays.


In [22]:
for i, val1 in enumerate(alphas): 
    for j, val2 in enumerate(sigmas):
        index = i * len(sigmas) + j
        print(index, '=',i, '*', len(sigmas), '+', j)

0 = 0 * 3 + 0
1 = 0 * 3 + 1
2 = 0 * 3 + 2
3 = 1 * 3 + 0
4 = 1 * 3 + 1
5 = 1 * 3 + 2
6 = 2 * 3 + 0
7 = 2 * 3 + 1
8 = 2 * 3 + 2


### Loop

In [28]:
for i, val1 in enumerate(alphas): 
    a.par.alpha = val1  #Set the parameter of outer loop
    for j, val2 in enumerate(sigmas):
        a.par.sigma = val2 #Set parameter of inner loop 
        index = i * len(sigmas) + j 
        HM[index] = a.solve_discrete().HM
        HF[index] = a.solve_discrete().HF

Home_ratios = HF/HM 
print(Home_ratios)

[0.54545455 0.30769231 0.21428571 1.         1.         1.
 1.83333333 3.25       4.66666667]
